In [38]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.transforms.autoaugment import AutoAugmentPolicy, AutoAugment

from torchsampler.imbalanced import ImbalancedDatasetSampler

train_dir = './train_images'
test_dir = './test_images'

transform = transforms.Compose(
    [transforms.AutoAugment(AutoAugmentPolicy.CIFAR10),
    transforms.Grayscale(),
    transforms.ToTensor(), 
    transforms.Normalize(mean=(0,),std=(1,))])

train_data = torchvision.datasets.ImageFolder(train_dir, transform=transform)
test_data = torchvision.datasets.ImageFolder(test_dir, transform=transform)

# 20% of the train dataset will be used as a validation exercise
valid_size = 0.2
batch_size = 32

num_train = len(train_data)
# (0, 1, 2, 3, ..., num_train)
indices_train = list(range(num_train))
# Reorders the indexes randomly (so we get 7, 4, 19...)
np.random.shuffle(indices_train)

# The first valid_size% of indexes will be for validation purposes
split_tv = int(np.floor(valid_size * num_train))

# Get the indexes, split between training and validation
train_new_idx, valid_idx = indices_train[split_tv:],indices_train[:split_tv]

train_sampler = ImbalancedDatasetSampler(train_data,train_new_idx)
valid_sampler = ImbalancedDatasetSampler(train_data,valid_idx)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=train_sampler, num_workers=1)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler, num_workers=1)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=1)
classes = ('noface','face')

In [39]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 18, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(18, 32, 5)
        self.fc1 = nn.Linear(32* 6 * 6, 48)
        self.fc2 = nn.Linear(48, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [40]:
import torch.optim as optim

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

network = Net()
network.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(network.parameters(), lr = 0.001, momentum=0.9)

In [41]:
epoch = 1
max_epoch = 12
print_every_n_batch = 400

best_network: Net = network
best_loss_validation = 999999999
steps_since_last_best = 0
threshold_early_stopping = 4 # Stop after 4 iterations without a new best network
best_model_found = False

print(device)

while not best_model_found and epoch <= max_epoch:

    total_loss_training = 0.0
    i = 0
    
    for data_training, target_training in train_loader:

        optimizer.zero_grad()
        inputs, labels = data_training.to(device), target_training.to(device)

        outputs = network(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss_training += loss.item()

        #Every 400 batches, we test the model on the validation data
        if i % print_every_n_batch == print_every_n_batch - 1:    # print every n mini-batches
            running_loss_training = total_loss_training / print_every_n_batch
            total_loss_training = 0.0
            total_loss_validation = 0.0


            # We feed the validation data to the network
            for data_valid, labels_valid in valid_loader:
                images, labels = data_valid.to(device), labels_valid.to(device)
                outputs = network(images)
                loss = criterion(outputs,labels)
                total_loss_validation += loss.item()
            
            running_loss_validation = total_loss_validation / len(valid_loader)

            print(epoch, i+1, "-", "Training loss: ", running_loss_training, " - Validation loss:", running_loss_validation)
            
            if(running_loss_validation < best_loss_validation):
                best_loss_validation = running_loss_validation
                best_network = network
                steps_since_last_best = 0
            else:
                steps_since_last_best += 1

            if steps_since_last_best >= threshold_early_stopping:
                best_model_found = True
                break

        i += 1

    epoch += 1

cpu
0 398 - Training loss:  0.6931013821065426  - Validation loss: 0.6929158996206543
0 798 - Training loss:  0.692806079685688  - Validation loss: 0.6925910791245903
0 1198 - Training loss:  0.6924880911409855  - Validation loss: 0.6923556515771753
0 1598 - Training loss:  0.6920529252290726  - Validation loss: 0.6916501319574562
0 1998 - Training loss:  0.691294252127409  - Validation loss: 0.6906279537320552
1 398 - Training loss:  0.6866407275199891  - Validation loss: 0.6827590084449755
1 798 - Training loss:  0.6730063183605671  - Validation loss: 0.6489783072720837
1 1198 - Training loss:  0.5660219924896955  - Validation loss: 0.45718664091846256
1 1598 - Training loss:  0.4131418836861849  - Validation loss: 0.36497460445875907
1 1998 - Training loss:  0.36606913324445484  - Validation loss: 0.3846292188618241
2 398 - Training loss:  0.3310965881496668  - Validation loss: 0.30338919496338956
2 798 - Training loss:  0.3027982799336314  - Validation loss: 0.28901700922109524
2 1

In [45]:
print(best_loss_validation)

0.0500475991905086


In [44]:
correct = 0
total = 0
with torch.no_grad():
    for imgs, labls in test_loader:
        images, labels = imgs.to(device), labls.to(device)
        outputs = best_network(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %2.2f %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 95.90 %
